In [1]:
import pandas as pd
from sklearn import metrics
import pickle

In [2]:
anime=pd.read_csv('anime.csv')
ratings=pd.read_csv('rating.csv')

In [3]:
ratings

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1
...,...,...,...
7813732,73515,16512,7
7813733,73515,17187,9
7813734,73515,22145,10
7813735,73516,790,9


In [4]:
anime=anime.drop(['genre','type','episodes','rating','members'],axis=1)

In [5]:
combined=pd.merge(anime,ratings)

In [6]:
combined.head()

,anime_id,name,user_id,rating
0,32281,Kimi no Na wa.,99,5
1,32281,Kimi no Na wa.,152,10
2,32281,Kimi no Na wa.,244,10
3,32281,Kimi no Na wa.,271,10
4,32281,Kimi no Na wa.,278,-1


In [7]:
user_ratings=combined.pivot_table(index=['user_id'],columns=['name'],values='rating')

In [8]:
user_ratings=user_ratings.dropna(thresh=50,axis=1).fillna(0)

In [9]:
user_ratings

name,&quot;Bungaku Shoujo&quot; Kyou no Oyatsu: Hatsukoi,&quot;Bungaku Shoujo&quot; Memoire,&quot;Bungaku Shoujo&quot; Movie,.hack//G.U. Returner,.hack//G.U. Trilogy,.hack//G.U. Trilogy: Parody Mode,.hack//Gift,.hack//Intermezzo,.hack//Liminality,.hack//Quantum,...,ef: A Tale of Memories.,ef: A Tale of Memories. - Prologue,ef: A Tale of Memories. - Recollections,iDOLM@STER Xenoglossia,s.CRY.ed,xxxHOLiC,xxxHOLiC Kei,xxxHOLiC Movie: Manatsu no Yoru no Yume,xxxHOLiC Rou,xxxHOLiC Shunmuki
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73512,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
73513,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
73514,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
item_similarity=metrics.pairwise.cosine_similarity(user_ratings.T)

In [11]:
item_similarity_df=pd.DataFrame(item_similarity,index=user_ratings.columns,columns=user_ratings.columns)

In [12]:
def get_similar_movies(movie_name,user_rating):
    similar_score=item_similarity_df[movie_name]*(user_rating)
    similar_score=similar_score.sort_values(ascending=False)
    return similar_score

In [13]:
action_lover=[('Pokemon',10)]

similar_movies=pd.DataFrame()

for movie,rating in action_lover:
    similar_movies=similar_movies.append(get_similar_movies(movie,rating))

/tmp/ipykernel_96731/1802781281.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similar_movies=similar_movies.append(get_similar_movies(movie,rating))


In [14]:
similar_movies.sum().sort_values(ascending=False)

name
Pokemon                                        5.000000
Pokemon Advanced Generation                    2.757661
Pokemon: Mewtwo no Gyakushuu                   2.335354
Pokemon: Kesshoutou no Teiou Entei             2.251684
Pokemon: Maboroshi no Pokemon Lugia Bakutan    2.243407
                                                 ...   
Mezame no Hakobune                             0.035451
Odin: Koushi Hansen Starlight                  0.034257
Mari Iyagi                                     0.034140
Soukou Kihei Votoms: Koei Futatabi             0.031187
Soukou Kihei Votoms: Kakuyaku taru Itan        0.024598
Length: 5651, dtype: float64

In [15]:
with open('item_similarity_df.pickle','wb') as f:
    pickle.dump(item_similarity_df, f)